
## Epoching and Artefact Rejection

In [1]:
import mne
import pandas as pd
import scipy.io
import os
import numpy as np
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import matplotlib.pyplot as plt
from datetime import datetime, date, time
from os.path import join as opj
from glob import glob
from pyprep.prep_pipeline import PrepPipeline
import time
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report
from autoreject import AutoReject


def ensure_dir(ed):
    import os
    try:
        os.makedirs(ed)
    except OSError:
        if not os.path.isdir(ed):
            raise


In [ ]:
'''
load preprocessed data
'''

# starting from a relative path /eeg_BIDS which you should also have
bids_root = '../eeg_BIDS/'

subs = [ name for name in os.listdir(bids_root) if name.startswith('sub') ]

n_jobs = 10

for subject in subs:
    
    print(subject)

    prepro_dir = opj(bids_root,'Preprocessed',subject)
    prepro_data = mne.io.read_raw_fif(glob(opj(prepro_dir, '*unf*'))[0], preload=True)



    # Filtering is needed before artefact rejection see: https://autoreject.github.io/stable/auto_examples/plot_autoreject_workflow.html#plot-autoreject-workflow
    cutoff_l = 0.1
    prepro_data_highpass = prepro_data.copy().filter(l_freq=cutoff_l, h_freq=None)

    events_dir = '/data04/Sebastian/EMP/events/'

    events_csv = pd.read_csv(glob(opj(events_dir,'EMP'+subject[-2:]+'*.csv'))[0])
    events = events_csv[['latency','trial','trigger']].to_numpy(dtype = int)

    new_events=mne.merge_events(events,range(1000,1999),1000, replace_events=True)
    new_events=mne.merge_events(new_events,range(2000,2999),2000, replace_events=True)

    #---------- Epoching and Artefact Rejection -------------#

    events_of_interest = {
        'manmade': 1000,
        'natural': 2000}

    # epoching the data to -200ms before stimulus onset to 500 ms after stimulus onset when the pictures disappears
    epoched_data = mne.Epochs(prepro_data_highpass, new_events, tmin=-0.2, tmax=0.5,baseline=None,
                        event_id =events_of_interest,
                        reject_by_annotation=False, preload=True)

    epoch_dir = opj(bids_root,'Epoched')
    ensure_dir(epoch_dir)
    epoched_data.save(opj(epoch_dir,subject+'-epo-ERP_RQ1_'+str(cutoff_l)+'_HP_.fif'), overwrite=True)


    ar = AutoReject(verbose='tqdm_notebook', n_jobs=n_jobs, random_state=234)
    epoched_data_clean_ar, rejection_log_fix = ar.fit_transform(epoched_data, return_log=True)

    autoreject_dir = opj(bids_root,'AutoReject')
    ensure_dir(autoreject_dir)
    epoched_data_clean_ar.save(opj(autoreject_dir,subject+'-epo-ERP_RQ1_autoreject_'+str(cutoff_l)+'_HP_.fif'), overwrite = True)
    # extract the indices of the dropped epochs and save them to a txt file

    dropped_epochs = list(np.where(rejection_log_fix.bad_epochs)[0])

    with open(opj(autoreject_dir,subject+'_droppedEpochs.txt'), 'w') as file:
        for x in dropped_epochs:
            file.write("%i\n" % x)

sub-001
Opening raw data file ../eeg_BIDS/Preprocessed/sub-001/sub-001_task_after_ica_raw_unfiltered.fif...


<ipython-input-3-d6a99bdeb11e>:17: RuntimeWarning: This filename (../eeg_BIDS/Preprocessed/sub-001/sub-001_task_after_ica_raw_unfiltered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  prepro_data = mne.io.read_raw_fif(glob(opj(prepro_dir, '*unf*'))[0], preload=True)


    Range : 0 ... 2541055 =      0.000 ...  4962.998 secs
Ready.
Reading 0 ... 2541055  =      0.000 ...  4962.998 secs...
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Filter length: 16897 samples (33.002 sec)

Not setting metadata
Not setting metadata
1200 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1200 events and 359 original time points ...
0 bad epochs dropped
Overwriting existing file.


<ipython-input-3-d6a99bdeb11e>:46: RuntimeWarning: This filename (../eeg_BIDS/Epoched/sub-001-epo-ERP_RQ1_0.1_HP_.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(opj(epoch_dir,subject+'-epo-ERP_RQ1_'+str(cutoff_l)+'_HP_.fif'), overwrite=True)
/home/speer/anaconda3/lib/python3.8/site-packages/autoreject/utils.py:66: UserWarning: 2 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


Running autoreject on ch_type=eeg


...

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/1200 [00:00<?,       ?it/s]

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/1200 [00:00<?,       ?it/s]

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]